In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 
%matplotlib inline

from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error

import xgboost as xgb

In [2]:
from sklearn import preprocessing

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

In [195]:
# df_events información de eventos realizado en la plataforma para un conjunto de usuarios hasta el 31/05/2018
df_events = pd.read_csv('../../data/events_up_to_01062018.csv',dtype=object)
# df_train archivo labels_training_set.csv indica para un subconjunto de los usuarios incluidos en el set de 
# eventos events_up_to_01062018.csv si los mismos realizaron una conversión (columna label = 1) o no (columna 
# label = 0) desde el 01/06/2018 hasta el 15/06/2018
df_labels = pd.read_csv('../../data/labels_training_set.csv',dtype=object)
# Personas a predecir conversion.
df_person = pd.read_csv('../../data/trocafone_kaggle_test.csv')

In [196]:
df_todas_las_personas = df_events[['person']].drop_duplicates()

## Descripcion basica de los set de datos con los que trabajo

In [197]:
df_events.groupby('event').agg(lambda x: x.isnull().sum()).T

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
timestamp,0,0,0,0,0,0,0,0,0,0,0
person,0,0,0,0,0,0,0,0,0,0,0
url,257,216312,65315,7091,160176,983,106406,130616,11201,1248124,204069
sku,191388,216312,0,0,160176,983,106406,130616,11201,0,204069
model,191388,216312,0,0,160176,0,106406,130616,11201,0,204069
condition,191388,216312,0,0,160176,983,106406,130616,11201,0,204069
storage,191388,216312,0,0,160176,983,106406,130616,11201,0,204069
color,191388,216312,0,0,160176,983,106406,130616,11201,0,204069
skus,187949,759,64155,6918,371,983,104196,28074,10940,1227318,204069
search_term,191388,216312,65315,7091,160176,983,106406,16853,11201,1248124,204069


In [198]:
df_events.describe()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
count,2341681,2341681,2341681,191131,1320530,1321513,1320530,1320530,1320530,505949,...,106406,204069,204069,204069,204069,204069,204069,204066,204069,204069
unique,1490912,11,38829,248,3693,208,5,8,63,52267,...,4,7,2,2206,122,51,4,393,131,366
top,2018-05-31 01:59:16,viewed product,c76b8417,/,2830.0,iPhone 6,Bom,16GB,Preto,"2820,6706,6720,2750,6649,7251,6663,12604,7224,...",...,Google,Paid,Returning,Unknown,Sao Paulo,Brazil,Smartphone,360x640,Windows 7,Chrome 66.0
freq,14,1248124,4438,64187,8462,107262,547617,442096,314925,2606,...,105195,91753,165827,36866,57304,197699,103502,73234,46648,57953


In [199]:
df_events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
df_labels.describe()

,person,label
count,19414,19414
unique,19414,2
top,b8f3eea9,0
freq,1,18434


In [201]:
df_labels.head(6)

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0
5,db2c4d27,1


In [202]:
df_labels.shape

(19414, 2)

In [203]:
df_labels['label'] = df_labels['label'].astype(int)
df_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19414 entries, 0 to 19413
Data columns (total 2 columns):
person    19414 non-null object
label     19414 non-null int64
dtypes: int64(1), object(1)
memory usage: 303.4+ KB


In [204]:
df_person.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


### | - Informacion enunciado tp2 - |

#### La información de estos archivos debe ser utilizada para entrenar un modelo de Machine Learning, de tal forma de poder indicar la probabilidad de que conjunto seleccionado de usuarios realice una conversión desde el 01/06/2018 al 15/06/2018. 
#### Se pedirá indicar esa probabilidad de conversión para usuarios que no se encuentran en el archivo df_labels_training_set.csv, pero para los cuales se cuenta con información en events_up_to_01062018.csv
#### Los grupos deberán probar distintos algoritmos de Machine Learning para predecir cuál es la probabilidad de conversión del conjunto de usuarios seleccionados de Trocafone para la competencia en el periodo descrito. A medida que los grupos realicen pruebas deben realizar el correspondiente submit en Kaggle para evaluar el resultado de los mismos.
#### Al finalizar la competencia el grupo que mejor resultado tenga obtendrá 10 puntos para cada uno de sus integrantes que podrán ser usados en el examen por promoción o segundo recuperatorio.


El TP2 se va a evaluar en función del siguiente criterio:

* Cantidad de trabajo (esfuerzo) del grupo: ¿Probaron muchos algoritmos? ¿Hicieron un buen trabajo de pre-procesamiento de los datos y feature engineering?
* Resultado obtenido en Kaggle (obviamente cuanto mejor resultado mejor nota)
* Presentación final del informe, calidad de la redacción, uso de información obtenida en el TP1, conclusiones presentadas.
* Performance de la solución final.


In [205]:
df_events["timestamp"] = pd.to_datetime(df_events["timestamp"])
df_events["month"] = df_events["timestamp"].dt.month
df_events["day"] = df_events["timestamp"].dt.day

In [206]:
df_events['week'] = df_events['timestamp'].dt.week

In [207]:
df_events["day_of_week"] = df_events['timestamp'].dt.weekday_name

In [208]:
viewed_product = df_events[df_events['event'] == 'viewed product']
brand_listing    = df_events[df_events['event'] == 'brand listing']
visited_site =  df_events[df_events['event'] == 'visited site']
ad_campaign_hit = df_events[df_events['event'] == 'ad campaign hit']
generic_listing = df_events[df_events['event'] == 'generic listing']
searched_products = df_events[df_events['event'] == 'searched products']
search_engine_hit = df_events[df_events['event'] == 'search engine hit']
checkout = df_events[df_events['event'] == 'checkout']
staticpage = df_events[df_events['event'] == 'staticpage']
conversion = df_events[df_events['event'] == 'conversion']
lead = df_events[df_events['event'] == 'lead']

In [209]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 27 columns):
timestamp                   datetime64[ns]
event                       object
person                      object
url                         object
sku                         object
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
month                       int64
day                         int64
week  

## event

#### Cantidad de veces que realiza cada evento.

In [210]:
df_events['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [211]:
df_cant_por_evento = pd.concat([pd.get_dummies(df_events['event']),df_events[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()
df_cant_por_evento.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
3,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0
4,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0


In [212]:
df_cant_por_evento.shape

(38829, 12)

#### Realiza el evento o no lo realiza

In [213]:
df_realiza_evento = pd.concat([df_events[['person']],pd.get_dummies(df_events['event'])],axis = 1)

In [214]:
df_realiza_evento = df_realiza_evento.drop_duplicates()

In [215]:
df_realiza_evento = df_realiza_evento.groupby('person').sum().reset_index()

In [216]:
df_realiza_evento.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,0008ed71,0,0,1,0,1,0,0,0,0,0,1
1,00091926,1,1,1,0,0,0,0,0,0,1,1
2,00091a7a,1,1,0,0,0,0,0,0,0,1,1
3,000ba417,1,1,1,1,1,0,1,0,0,1,1
4,000c79fe,1,0,1,0,1,0,1,1,0,1,1


In [217]:
df_realiza_evento.shape

(38829, 12)

## model

### model - viewed product

#### Cantidad de visitas de c/usuario a los modelos.

Se llenan con ceros los valores nulos ya que estos se deben a que un usuario no hizo un evento viewed product, es
decir no visito la pagina de un producto, entonces es correcto poner un 0.

In [218]:
top_model = ['iPhone 6S Plus', 'iPhone 7 Plus', 'iPhone 6 Plus', 'iPhone 6S',
       'Samsung Galaxy S7 Edge', 'Samsung Galaxy S6 Flat', 'iPhone SE',
       'Samsung Galaxy S6 Edge', 'iPhone 5s', 'Samsung Galaxy S5',
       'Samsung Galaxy J5', 'iPhone 6', 'Samsung Galaxy A7 2017', 'iPhone 5c',
       'Samsung Galaxy J7 2016 Metal']

# Elegidos en base a analisis del df con labels (ver notebook).

In [219]:
df_visitas_a_modelos = pd.concat([viewed_product['person'],pd.get_dummies(viewed_product['model'])[top_model]],axis =1)\
            .groupby('person').sum().reset_index()

In [220]:
df_visitas_a_modelos = pd.merge(df_todas_las_personas,df_visitas_a_modelos,on = 'person', how = 'left')

In [221]:
df_visitas_a_modelos = df_visitas_a_modelos.fillna(0)

In [222]:
nombres_columnas = ['vis_'+columna for columna in df_visitas_a_modelos.columns]
nombres_columnas[0] = 'person'
df_visitas_a_modelos.columns = nombres_columnas

In [223]:
df_visitas_a_modelos.head()

,person,vis_iPhone 6S Plus,vis_iPhone 7 Plus,vis_iPhone 6 Plus,vis_iPhone 6S,vis_Samsung Galaxy S7 Edge,vis_Samsung Galaxy S6 Flat,vis_iPhone SE,vis_Samsung Galaxy S6 Edge,vis_iPhone 5s,vis_Samsung Galaxy S5,vis_Samsung Galaxy J5,vis_iPhone 6,vis_Samsung Galaxy A7 2017,vis_iPhone 5c,vis_Samsung Galaxy J7 2016 Metal
0,4886f805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0297fc1e,12.0,9.0,10.0,113.0,2.0,0.0,8.0,0.0,1.0,0.0,0.0,213.0,0.0,0.0,0.0
3,2d681dd8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cccea85e,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0


#### Visita o no un modelo.

In [224]:
df_visitas_a_modelos2 = pd.concat([viewed_product['person'],pd.get_dummies(viewed_product['model'])[top_model]],axis =1)\
            .drop_duplicates().groupby('person').sum().reset_index()

In [225]:
df_visitas_a_modelos2 = pd.merge(df_todas_las_personas,df_visitas_a_modelos2,on = 'person', how = 'left')

In [226]:
df_visitas_a_modelos2 = df_visitas_a_modelos2.fillna(0)

In [227]:
df_visitas_a_modelos2.columns = df_visitas_a_modelos.columns

In [228]:
df_visitas_a_modelos2.head()

,person,vis_iPhone 6S Plus,vis_iPhone 7 Plus,vis_iPhone 6 Plus,vis_iPhone 6S,vis_Samsung Galaxy S7 Edge,vis_Samsung Galaxy S6 Flat,vis_iPhone SE,vis_Samsung Galaxy S6 Edge,vis_iPhone 5s,vis_Samsung Galaxy S5,vis_Samsung Galaxy J5,vis_iPhone 6,vis_Samsung Galaxy A7 2017,vis_iPhone 5c,vis_Samsung Galaxy J7 2016 Metal
0,4886f805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0297fc1e,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2d681dd8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cccea85e,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


### model - checkout

#### Cantidad de checkouts a modelos

Tiene sentido remplazar valores nulos por 0s.

In [229]:
top_model = ['iPhone 6S', 'Samsung Galaxy J5', 'Samsung Galaxy J7 Prime',
       'iPhone 7 Plus', 'iPhone 4G', 'Samsung Galaxy On 7',
       'Motorola Moto X Play 4G Dual', 'Samsung Galaxy J7 2016 Metal',
       'iPhone 7', 'Motorola Moto X2', 'Samsung Galaxy S7', 'iPhone 4S',
       'Lenovo Vibe A7010 Dual Chip', 'Motorola Moto G4 Plus', 'iPhone 5s']

In [230]:
df_checkouts_modelos = pd.concat([checkout['person'],pd.get_dummies(checkout['model'])[top_model]],axis =1)\
            .groupby('person').sum().reset_index()

In [231]:
df_checkouts_modelos = pd.merge(df_todas_las_personas,df_checkouts_modelos,on = 'person', how = 'left')

In [232]:
df_checkouts_modelos = df_checkouts_modelos.fillna(0)

In [233]:
nombres_columnas = ['checkout_'+columna for columna in df_checkouts_modelos.columns]
nombres_columnas[0] = 'person'
df_checkouts_modelos.columns = nombres_columnas

In [234]:
df_checkouts_modelos.head()

,person,checkout_iPhone 6S,checkout_Samsung Galaxy J5,checkout_Samsung Galaxy J7 Prime,checkout_iPhone 7 Plus,checkout_iPhone 4G,checkout_Samsung Galaxy On 7,checkout_Motorola Moto X Play 4G Dual,checkout_Samsung Galaxy J7 2016 Metal,checkout_iPhone 7,checkout_Motorola Moto X2,checkout_Samsung Galaxy S7,checkout_iPhone 4S,checkout_Lenovo Vibe A7010 Dual Chip,checkout_Motorola Moto G4 Plus,checkout_iPhone 5s
0,4886f805,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0297fc1e,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2d681dd8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,cccea85e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


#### Hizo un checkout o no a un modelo.

In [235]:
df_checkouts_modelos2 = pd.concat([checkout['person'],pd.get_dummies(checkout['model'])[top_model]],axis =1)\
            .drop_duplicates().groupby('person').sum().reset_index()

In [236]:
df_checkouts_modelos2 = pd.merge(df_todas_las_personas,df_checkouts_modelos2,on = 'person', how = 'left')

In [237]:
df_checkouts_modelos2 = df_checkouts_modelos2.fillna(0)

In [238]:
df_checkouts_modelos2.columns = df_checkouts_modelos.columns

In [239]:
df_checkouts_modelos2.head()

,person,checkout_iPhone 6S,checkout_Samsung Galaxy J5,checkout_Samsung Galaxy J7 Prime,checkout_iPhone 7 Plus,checkout_iPhone 4G,checkout_Samsung Galaxy On 7,checkout_Motorola Moto X Play 4G Dual,checkout_Samsung Galaxy J7 2016 Metal,checkout_iPhone 7,checkout_Motorola Moto X2,checkout_Samsung Galaxy S7,checkout_iPhone 4S,checkout_Lenovo Vibe A7010 Dual Chip,checkout_Motorola Moto G4 Plus,checkout_iPhone 5s
0,4886f805,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0297fc1e,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2d681dd8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,cccea85e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## color

### color - viewed product

#### Cantidad de visitas a productos por color.

In [240]:
top_color = ['Cinza espacial', 'Preto', 'Branco', 'Prateado', 'Ouro Rosa', 'Prata',
       'Rosa', 'Dourado', 'Azul', 'Ametista', 'Cinza', 'Preto Matte', 'Verde',
       'Vermelho', 'Preto Brilhante']

In [241]:
df_color_viewedproduct = pd.concat([viewed_product['person'],pd.get_dummies(viewed_product['color'])[top_color]],axis =1)\
            .groupby('person').sum().reset_index()

In [242]:
df_color_viewedproduct = pd.merge(df_todas_las_personas,df_color_viewedproduct,on = 'person', how = 'left')

In [243]:
df_color_viewedproduct = df_color_viewedproduct.fillna(0)

In [244]:
nombres_columnas = ['viewedproduct_'+columna for columna in df_color_viewedproduct.columns]
nombres_columnas[0] = 'person'
df_color_viewedproduct.columns = nombres_columnas

In [245]:
df_color_viewedproduct.head()

,person,viewedproduct_Cinza espacial,viewedproduct_Preto,viewedproduct_Branco,viewedproduct_Prateado,viewedproduct_Ouro Rosa,viewedproduct_Prata,viewedproduct_Rosa,viewedproduct_Dourado,viewedproduct_Azul,viewedproduct_Ametista,viewedproduct_Cinza,viewedproduct_Preto Matte,viewedproduct_Verde,viewedproduct_Vermelho,viewedproduct_Preto Brilhante
0,4886f805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,8.0,0.0,1.0,6.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0297fc1e,86.0,2.0,0.0,76.0,85.0,0.0,1.0,130.0,0.0,0.0,0.0,14.0,0.0,1.0,8.0
3,2d681dd8,0.0,4.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,cccea85e,0.0,204.0,122.0,1.0,1.0,8.0,8.0,32.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


#### Visita producto por color 

In [246]:
df_color_viewedproduct2 = pd.concat([viewed_product['person'],pd.get_dummies(viewed_product['color'])[top_color]],axis =1)\
            .drop_duplicates().groupby('person').sum().reset_index()

In [247]:
df_color_viewedproduct2 = pd.merge(df_todas_las_personas,df_color_viewedproduct2,on = 'person', how = 'left')

In [248]:
df_color_viewedproduct2 = df_color_viewedproduct2.fillna(0)

In [249]:
df_color_viewedproduct2.columns = df_color_viewedproduct.columns

In [250]:
df_color_viewedproduct2.head()

,person,viewedproduct_Cinza espacial,viewedproduct_Preto,viewedproduct_Branco,viewedproduct_Prateado,viewedproduct_Ouro Rosa,viewedproduct_Prata,viewedproduct_Rosa,viewedproduct_Dourado,viewedproduct_Azul,viewedproduct_Ametista,viewedproduct_Cinza,viewedproduct_Preto Matte,viewedproduct_Verde,viewedproduct_Vermelho,viewedproduct_Preto Brilhante
0,4886f805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0297fc1e,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
3,2d681dd8,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,cccea85e,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## storage

### storage - viewed product

#### Cantidad de visitas a productos por storage

In [251]:
df_storage_viewedproduct = pd.concat([viewed_product['person'],pd.get_dummies(viewed_product['storage'])],axis =1)\
            .groupby('person').sum().reset_index()

In [252]:
df_storage_viewedproduct = pd.merge(df_todas_las_personas,df_storage_viewedproduct,on = 'person', how = 'left')

In [253]:
df_storage_viewedproduct = df_storage_viewedproduct.fillna(0)

In [254]:
nombres_columnas = ['viewedproduct_'+columna for columna in df_storage_viewedproduct.columns]
nombres_columnas[0] = 'person'
df_storage_viewedproduct.columns = nombres_columnas

In [255]:
df_storage_viewedproduct.head()

,person,viewedproduct_128GB,viewedproduct_16GB,viewedproduct_256GB,viewedproduct_32GB,viewedproduct_4GB,viewedproduct_512MB,viewedproduct_64GB,viewedproduct_8GB
0,4886f805,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
1,ad93850f,1.0,10.0,0.0,7.0,0.0,0.0,2.0,0.0
2,0297fc1e,50.0,126.0,6.0,33.0,0.0,0.0,189.0,0.0
3,2d681dd8,6.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0
4,cccea85e,0.0,85.0,0.0,597.0,0.0,0.0,32.0,25.0


#### Visita producto por storage

In [256]:
df_storage_viewedproduct2 = pd.concat([viewed_product['person'],pd.get_dummies(viewed_product['storage'])],axis =1)\
            .drop_duplicates().groupby('person').sum().reset_index()

In [257]:
df_storage_viewedproduct2 = pd.merge(df_todas_las_personas,df_storage_viewedproduct2,on = 'person', how = 'left')

In [258]:
df_storage_viewedproduct2 = df_storage_viewedproduct2.fillna(0)

In [259]:
df_storage_viewedproduct2.columns = df_storage_viewedproduct.columns

In [260]:
df_storage_viewedproduct2.head()

,person,viewedproduct_128GB,viewedproduct_16GB,viewedproduct_256GB,viewedproduct_32GB,viewedproduct_4GB,viewedproduct_512MB,viewedproduct_64GB,viewedproduct_8GB
0,4886f805,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,ad93850f,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0297fc1e,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
3,2d681dd8,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,cccea85e,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0


## condition

### condition - viewedproduct

#### Cantidad de visitas a productos por condition

In [261]:
df_condition_viewedproduct = pd.concat([viewed_product['person'],pd.get_dummies(viewed_product['condition'])],axis =1)\
            .groupby('person').sum().reset_index()

In [262]:
df_condition_viewedproduct = pd.merge(df_todas_las_personas,df_condition_viewedproduct,on = 'person', how = 'left')

In [263]:
df_condition_viewedproduct = df_condition_viewedproduct.fillna(0)

In [264]:
nombres_columnas = ['viewedproduct_'+columna for columna in df_condition_viewedproduct.columns]
nombres_columnas[0] = 'person'
df_condition_viewedproduct.columns = nombres_columnas

In [265]:
df_condition_viewedproduct.head()

,person,viewedproduct_Bom,viewedproduct_Bom - Sem Touch ID,viewedproduct_Excelente,viewedproduct_Muito Bom,viewedproduct_Novo
0,4886f805,0.0,0.0,3.0,1.0,0.0
1,ad93850f,2.0,8.0,4.0,6.0,0.0
2,0297fc1e,239.0,2.0,46.0,117.0,0.0
3,2d681dd8,13.0,0.0,0.0,0.0,0.0
4,cccea85e,200.0,1.0,191.0,346.0,1.0


#### Visita producto por condition

In [266]:
df_condition_viewedproduct2 = pd.concat([viewed_product['person'],pd.get_dummies(viewed_product['condition'])],axis =1)\
            .drop_duplicates().groupby('person').sum().reset_index()

In [267]:
df_condition_viewedproduct2 = pd.merge(df_todas_las_personas,df_condition_viewedproduct2,on = 'person', how = 'left')

In [268]:
df_condition_viewedproduct2 = df_condition_viewedproduct2.fillna(0)

In [269]:
df_condition_viewedproduct2.columns = df_condition_viewedproduct.columns

In [270]:
df_condition_viewedproduct2.head()

,person,viewedproduct_Bom,viewedproduct_Bom - Sem Touch ID,viewedproduct_Excelente,viewedproduct_Muito Bom,viewedproduct_Novo
0,4886f805,0.0,0.0,1.0,1.0,0.0
1,ad93850f,1.0,1.0,1.0,1.0,0.0
2,0297fc1e,1.0,1.0,1.0,1.0,0.0
3,2d681dd8,1.0,0.0,0.0,0.0,0.0
4,cccea85e,1.0,1.0,1.0,1.0,1.0


## new vs returning

### new vs returning - visited site

Analice las personas para ver cuando tomaban los valores new o returning y no puedo predecirlos, por lo tanto los valores nulos los voy a llenar con la moda.

### Volvio la persona

In [318]:
df_newvsreturning_visitedsite = pd.concat([visited_site['person'],pd.get_dummies(visited_site['new_vs_returning'])],axis =1)\
            .drop_duplicates().groupby('person').sum().reset_index()

df_newvsreturning_visitedsite['vuelve'] = df_newvsreturning_visitedsite['New']*df_newvsreturning_visitedsite['Returning']

df_newvsreturning_visitedsite = df_newvsreturning_visitedsite.drop(columns = ['New','Returning'])

In [375]:
df_newvsreturning_visitedsite.head()

,person,vuelve
0,0008ed71,1
1,00091926,1
2,00091a7a,0
3,000ba417,1
4,000c79fe,0


Para las personas sin visited site, considero que vuelven si entran en aunque sea 2 dias distintos.

In [341]:
personas_con_visited_site = visited_site[['person','event']].drop_duplicates()

In [342]:
personas_sin_visited_site = pd.merge(df_todas_las_personas,personas_con_visited_site,on = 'person', how = 'left')

In [343]:
personas_sin_visited_site = personas_sin_visited_site[personas_sin_visited_site['event'] != 'visited site'][['person']]

In [344]:
personas_sin_visited_site.head()

,person
22,d21b8e6b
64,566b70c7
584,37a0837e
1263,62816be1
1714,77cc1a7e


In [345]:
personas_sin_visited_site = pd.merge(df_events[['person','day']],personas_sin_visited_site,on = 'person',how = 'inner')

In [349]:
personas_sin_visited_site = pd.concat([personas_sin_visited_site['person'],pd.get_dummies(personas_sin_visited_site['day'])],axis = 1)\
                            .drop_duplicates().groupby('person').sum().reset_index()

In [364]:
personas_sin_visited_site['suma'] = personas_sin_visited_site.sum(axis = 1)

In [367]:
personas_sin_visited_site['vuelve'] = (personas_sin_visited_site['suma'] > 1).astype(int)

In [369]:
personas_sin_visited_site = personas_sin_visited_site[['person','vuelve']]

In [376]:
personas_sin_visited_site.head()

,person,vuelve
0,001b0bf9,0
1,01118279,0
2,01127f1d,0
3,02ba82a5,0
4,02ecfb15,0


In [377]:
df_newvsreturning_visitedsite = pd.concat([df_newvsreturning_visitedsite,personas_sin_visited_site])

In [381]:
df_newvsreturning_visitedsite.shape

(38829, 2)

In [382]:
df_newvsreturning_visitedsite.head()

,person,vuelve
0,0008ed71,1
1,00091926,1
2,00091a7a,0
3,000ba417,1
4,000c79fe,0


## region

### region - visited site

#### Visita desde una region de Brazil

In [430]:
estados_br = visited_site[visited_site['country'] == 'Brazil']['region'].value_counts().index
estados_br

Index(['Sao Paulo', 'Unknown', 'Minas Gerais', 'Rio de Janeiro', 'Bahia',
       'Pernambuco', 'Ceara', 'Parana', 'Rio Grande do Sul', 'Espirito Santo',
       'Federal District', 'Maranhao', 'Goias', 'Santa Catarina', 'Para',
       'Rio Grande do Norte', 'Paraíba', 'Piaui', 'Alagoas', 'Sergipe',
       'Amazonas', 'Mato Grosso', 'Tocantins', 'Mato Grosso do Sul',
       'Rondonia', 'Amapa', 'Acre', 'Roraima'],
      dtype='object')

In [431]:
region_mas_popular =  visited_site['region'].value_counts().head(1)
region_mas_popular = region_mas_popular.index[0]
region_mas_popular

'Sao Paulo'

In [432]:
df_region_visitedsite = pd.concat([visited_site['person'],pd.get_dummies(visited_site['region'])[estados_br]],axis =1)\
            .drop_duplicates().groupby('person').sum().reset_index()

In [433]:
df_region_visitedsite = pd.merge(df_todas_las_personas,df_region_visitedsite,on = 'person', how = 'left')

In [434]:
df_region_visitedsite.head()

,person,Sao Paulo,Unknown,Minas Gerais,Rio de Janeiro,Bahia,Pernambuco,Ceara,Parana,Rio Grande do Sul,...,Alagoas,Sergipe,Amazonas,Mato Grosso,Tocantins,Mato Grosso do Sul,Rondonia,Amapa,Acre,Roraima
0,4886f805,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0297fc1e,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2d681dd8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cccea85e,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [435]:
df_region_visitedsite = df_region_visitedsite.fillna({region_mas_popular:1})
df_region_visitedsite = df_region_visitedsite.fillna(0)
nombres_columnas = ['visitedsite_'+columna for columna in df_region_visitedsite.columns]
nombres_columnas[0] = 'person'
df_region_visitedsite.columns = nombres_columnas
df_region_visitedsite.rename(columns = {'visitedsite_Unknown':'visitedsite_region desconocida'},inplace = True)

In [436]:
df_region_visitedsite.head()

,person,visitedsite_Sao Paulo,visitedsite_region desconocida,visitedsite_Minas Gerais,visitedsite_Rio de Janeiro,visitedsite_Bahia,visitedsite_Pernambuco,visitedsite_Ceara,visitedsite_Parana,visitedsite_Rio Grande do Sul,...,visitedsite_Alagoas,visitedsite_Sergipe,visitedsite_Amazonas,visitedsite_Mato Grosso,visitedsite_Tocantins,visitedsite_Mato Grosso do Sul,visitedsite_Rondonia,visitedsite_Amapa,visitedsite_Acre,visitedsite_Roraima
0,4886f805,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0297fc1e,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2d681dd8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cccea85e,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## device type

### device type - visited site

#### Visita desde un dispositivo

In [445]:
dispositivo_mas_popular = visited_site['device_type'].value_counts().head(1).index[0]
dispositivo_mas_popular

'Smartphone'

In [439]:
df_devicetype_visitedsite = pd.concat([visited_site['person'],pd.get_dummies(visited_site['device_type'])],axis =1)\
            .drop_duplicates().groupby('person').sum().reset_index()

In [440]:
df_devicetype_visitedsite = pd.merge(df_todas_las_personas,df_devicetype_visitedsite,on = 'person', how = 'left')

In [442]:
df_devicetype_visitedsite.rename(columns = {'Unknown' : 'device desconocido'}, inplace = True)

In [446]:
df_devicetype_visitedsite = df_devicetype_visitedsite.fillna({dispositivo_mas_popular:1})
df_devicetype_visitedsite = df_devicetype_visitedsite.fillna(0)
nombres_columnas = ['visitedsite_'+columna for columna in df_devicetype_visitedsite.columns]
nombres_columnas[0] = 'person'
df_devicetype_visitedsite.columns = nombres_columnas

In [450]:
df_devicetype_visitedsite.head()

,person,visitedsite_Computer,visitedsite_Smartphone,visitedsite_Tablet,visitedsite_device desconocido
0,4886f805,0.0,1.0,0.0,0.0
1,ad93850f,0.0,1.0,0.0,0.0
2,0297fc1e,0.0,1.0,0.0,0.0
3,2d681dd8,1.0,0.0,0.0,0.0
4,cccea85e,1.0,0.0,0.0,0.0


## screen resolution

### screen resolution - visited site

In [465]:
top_screenresolution = ['360x640', '1366x768', '320x570', '1600x900', '320x534', '1280x1024',
       '412x732', '375x667', '414x736', '320x490', '1536x864', '320x569',
       '1920x1080', '1024x768', '1324x745']

In [466]:
resolucion_mas_pouplar = visited_site['screen_resolution'].value_counts().head(1).index[0]

In [467]:
df_screenresolution_visitedsite = pd.concat([visited_site['person'],pd.get_dummies(visited_site['screen_resolution'])[top_screenresolution]],axis =1)\
            .drop_duplicates().groupby('person').sum().reset_index()

In [468]:
df_screenresolution_visitedsite = pd.merge(df_todas_las_personas,df_screenresolution_visitedsite,on = 'person', how = 'left')

In [469]:
df_screenresolution_visitedsite = df_screenresolution_visitedsite.fillna({resolucion_mas_pouplar:1})
df_screenresolution_visitedsite = df_screenresolution_visitedsite.fillna(0)
nombres_columnas = ['visitedsite_'+columna for columna in df_screenresolution_visitedsite.columns]
nombres_columnas[0] = 'person'
df_screenresolution_visitedsite.columns = nombres_columnas

In [471]:
df_screenresolution_visitedsite.head()

,person,visitedsite_360x640,visitedsite_1366x768,visitedsite_320x570,visitedsite_1600x900,visitedsite_320x534,visitedsite_1280x1024,visitedsite_412x732,visitedsite_375x667,visitedsite_414x736,visitedsite_320x490,visitedsite_1536x864,visitedsite_320x569,visitedsite_1920x1080,visitedsite_1024x768,visitedsite_1324x745
0,4886f805,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0297fc1e,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2d681dd8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cccea85e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## operating system version

### operating system version - visited site

In [552]:
top_operatingsystemversion = ['Android 7', 'Windows 7 ', 'Windows 10 ', 'Android 5.1',
       'Android 6.0.1', 'Android 7.1.1', 'Android 5.0.2', 'Android 6',
       'Android 4.4.2', 'iOS 11.2.6', 'Windows Phone 8.1', 'Android 4.1.2',
       'Windows Vista ', 'iOS 10.3.3', 'Android 8']

In [553]:
operatingsystemversion_mas_popular = visited_site['operating_system_version'].value_counts().head(1).index[0]

In [554]:
df_operatingsystemversion_visitedsite = pd.concat([visited_site['person'],pd.get_dummies(visited_site['operating_system_version'])[top_operatingsystemversion]],axis =1)\
            .drop_duplicates().groupby('person').sum().reset_index()

In [555]:
df_operatingsystemversion_visitedsite = pd.merge(df_todas_las_personas,df_operatingsystemversion_visitedsite,on = 'person', how = 'left')

In [556]:
df_operatingsystemversion_visitedsite = df_operatingsystemversion_visitedsite.fillna({operatingsystemversion_mas_popular:1})
df_operatingsystemversion_visitedsite = df_operatingsystemversion_visitedsite.fillna(0)
nombres_columnas = ['visitedsite_'+columna for columna in df_operatingsystemversion_visitedsite.columns]
nombres_columnas[0] = 'person'
df_operatingsystemversion_visitedsite.columns = nombres_columnas

In [597]:
df_operatingsystemversion_visitedsite.head()

,person,visitedsite_Android 7,visitedsite_Windows 7,visitedsite_Windows 10,visitedsite_Android 5.1,visitedsite_Android 6.0.1,visitedsite_Android 7.1.1,visitedsite_Android 5.0.2,visitedsite_Android 6,visitedsite_Android 4.4.2,visitedsite_iOS 11.2.6,visitedsite_Windows Phone 8.1,visitedsite_Android 4.1.2,visitedsite_Windows Vista,visitedsite_iOS 10.3.3,visitedsite_Android 8
0,4886f805,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0297fc1e,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2d681dd8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cccea85e,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## browser version

### browser version - visited site

In [539]:
top_browserversion = ['Chrome Mobile 66.0', 'Chrome 65.0', 'Chrome 66.0', 'Chrome 64.0',
       'Chrome 63.0', 'Chrome Mobile 65.0', 'Chrome Mobile 64.0',
       'Chrome Mobile 39', 'Chrome Mobile 63.0', 'Samsung Internet 3.3',
       'Samsung Internet 6.4', 'Mobile Safari 11', 'IE Mobile 11',
       'Edge 16.16299', 'Chrome Mobile 36.0']

In [540]:
browserversion_mas_popular = visited_site['browser_version'].value_counts().head(1).index[0]

In [541]:
df_browserversion_visitedsite = pd.concat([visited_site['person'],pd.get_dummies(visited_site['browser_version'])[top_browserversion]],axis =1)\
            .drop_duplicates().groupby('person').sum().reset_index()

In [542]:
df_browserversion_visitedsite = pd.merge(df_todas_las_personas,df_browserversion_visitedsite,on = 'person', how = 'left')

In [543]:
df_browserversion_visitedsite = df_browserversion_visitedsite.fillna({browserversion_mas_popular:1})
df_browserversion_visitedsite = df_browserversion_visitedsite.fillna(0)
nombres_columnas = ['visitedsite_'+columna for columna in df_browserversion_visitedsite.columns]
nombres_columnas[0] = 'person'
df_browserversion_visitedsite.columns = nombres_columnas

In [544]:
df_browserversion_visitedsite.head()

,person,visitedsite_Chrome Mobile 66.0,visitedsite_Chrome 65.0,visitedsite_Chrome 66.0,visitedsite_Chrome 64.0,visitedsite_Chrome 63.0,visitedsite_Chrome Mobile 65.0,visitedsite_Chrome Mobile 64.0,visitedsite_Chrome Mobile 39,visitedsite_Chrome Mobile 63.0,visitedsite_Samsung Internet 3.3,visitedsite_Samsung Internet 6.4,visitedsite_Mobile Safari 11,visitedsite_IE Mobile 11,visitedsite_Edge 16.16299,visitedsite_Chrome Mobile 36.0
0,4886f805,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0297fc1e,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2d681dd8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cccea85e,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## campaign source

### campaign source - ad campaign hit

#### Cantidad de visitas desde una campania publicitaria

In [582]:
# Parece tener sentido llenar los valores nulos con 0s.

In [588]:
top_campaignsource = ['rtbhouse', 'criteo', 'google', 'zanox', 'manifest', 'bing', 'afilio',
       'buscape', 'rakuten', 'FacebookAds', 'indexa', 'datacrush', 'emblue',
       'blog', 'yotpo']

In [589]:
campaignsource_mas_popular = ad_campaign_hit['campaign_source'].value_counts().head(1).index[0]
campaignsource_mas_popular

'google'

In [590]:
df_campaignsource_adcampaignhit = pd.concat([ad_campaign_hit['person'],pd.get_dummies(ad_campaign_hit['campaign_source'])[top_campaignsource]],axis =1)\
            .groupby('person').sum().reset_index()

In [591]:
df_campaignsource_adcampaignhit = pd.merge(df_todas_las_personas,df_campaignsource_adcampaignhit,on = 'person', how = 'left')

In [592]:
df_campaignsource_adcampaignhit = df_campaignsource_adcampaignhit.fillna(0)
nombres_columnas = ['adcampaignhit_'+columna for columna in df_campaignsource_adcampaignhit.columns]
nombres_columnas[0] = 'person'
df_campaignsource_adcampaignhit.columns = nombres_columnas

In [595]:
df_campaignsource_adcampaignhit.head()

,person,adcampaignhit_rtbhouse,adcampaignhit_criteo,adcampaignhit_google,adcampaignhit_zanox,adcampaignhit_manifest,adcampaignhit_bing,adcampaignhit_afilio,adcampaignhit_buscape,adcampaignhit_rakuten,adcampaignhit_FacebookAds,adcampaignhit_indexa,adcampaignhit_datacrush,adcampaignhit_emblue,adcampaignhit_blog,adcampaignhit_yotpo
0,4886f805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,0.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0297fc1e,18.0,5.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2d681dd8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cccea85e,1.0,2.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# ---------------------------------------------------------------------

In [599]:
df_train = pd.merge(df_cant_por_evento, df_visitas_a_modelos,on = 'person',how = 'inner')
df_train = pd.merge(df_train, df_checkouts_modelos,on = 'person',how = 'inner')
df_train = pd.merge(df_train, df_color_viewedproduct,on = 'person',how = 'inner')
df_train = pd.merge(df_train, df_storage_viewedproduct,on = 'person',how = 'inner')
df_train = pd.merge(df_train, df_condition_viewedproduct,on = 'person',how = 'inner')
df_train = pd.merge(df_train, df_newvsreturning_visitedsite,on = 'person',how = 'inner')
df_train = pd.merge(df_train, df_region_visitedsite,on = 'person',how = 'inner')
df_train = pd.merge(df_train, df_devicetype_visitedsite,on = 'person',how = 'inner')
df_train = pd.merge(df_train, df_screenresolution_visitedsite,on = 'person',how = 'inner')
df_train = pd.merge(df_train, df_operatingsystemversion_visitedsite,on = 'person',how = 'inner')
df_train = pd.merge(df_train, df_browserversion_visitedsite,on = 'person',how = 'inner')
df_train = pd.merge(df_train, df_campaignsource_adcampaignhit,on = 'person',how = 'inner')

In [602]:
df_train.head()

,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,adcampaignhit_bing,adcampaignhit_afilio,adcampaignhit_buscape,adcampaignhit_rakuten,adcampaignhit_FacebookAds,adcampaignhit_indexa,adcampaignhit_datacrush,adcampaignhit_emblue,adcampaignhit_blog,adcampaignhit_yotpo
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [601]:
list(df_train.columns)

['person',
 'ad campaign hit',
 'brand listing',
 'checkout',
 'conversion',
 'generic listing',
 'lead',
 'search engine hit',
 'searched products',
 'staticpage',
 'viewed product',
 'visited site',
 'vis_iPhone 6S Plus',
 'vis_iPhone 7 Plus',
 'vis_iPhone 6 Plus',
 'vis_iPhone 6S',
 'vis_Samsung Galaxy S7 Edge',
 'vis_Samsung Galaxy S6 Flat',
 'vis_iPhone SE',
 'vis_Samsung Galaxy S6 Edge',
 'vis_iPhone 5s',
 'vis_Samsung Galaxy S5',
 'vis_Samsung Galaxy J5',
 'vis_iPhone 6',
 'vis_Samsung Galaxy A7 2017',
 'vis_iPhone 5c',
 'vis_Samsung Galaxy J7 2016 Metal',
 'checkout_iPhone 6S',
 'checkout_Samsung Galaxy J5',
 'checkout_Samsung Galaxy J7 Prime',
 'checkout_iPhone 7 Plus',
 'checkout_iPhone 4G',
 'checkout_Samsung Galaxy On 7',
 'checkout_Motorola Moto X Play 4G Dual',
 'checkout_Samsung Galaxy J7 2016 Metal',
 'checkout_iPhone 7',
 'checkout_Motorola Moto X2',
 'checkout_Samsung Galaxy S7',
 'checkout_iPhone 4S',
 'checkout_Lenovo Vibe A7010 Dual Chip',
 'checkout_Motorola Moto 

In [606]:
df_train.to_csv("Santi_FeaturesConEventos.csv")